In [20]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import udf
from pyspark.mllib.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.param import Param, Params
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.stat import Statistics
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from IPython.display import display
from ipywidgets import interact
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from operator import add
import sys
import numpy as np
import pandas as pd
import time
import datetime

# 1. load data
## 1.1 big trainset

In [3]:
# data=spark.read.csv('data/train_flight.csv',header=True,inferSchema=True)
# data.printSchema()

root
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (nullable = true)
 |-- schedule_departure: integer (nullable = true)
 |-- NEW_DAY: integer (nullable = true)



## 1.2 small_trainset

In [ ]:
# data1.write.option("header", "true").csv("data/small.csv")
# print("OK")
data_small=spark.read.csv('data/small.csv',header=True,inferSchema=True)
data_small.count()

In [26]:
data_small.printSchema()

root
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (nullable = true)
 |-- schedule_departure: integer (nullable = true)
 |-- NEW_DAY: integer (nullable = true)



# 2 feature transformation pipeline
## 2.1 feature selection (can be updated)

In [ ]:
 # can be updated
dataset=data_small
# dataset=data
categoricalColumns = []  # to add
numericCols = ['DEPARTURE_TIME','NEW_DAY']  # to add


## 2.2 transform and onehot

In [60]:

from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
cols=dataset.columns

stages = [] 
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, 
        outputCol=categoricalCol+"Index")
    encoder = OneHotEncoder(inputCol=categoricalCol+"Index", 
        outputCol=categoricalCol+"classVec")
    stages += [stringIndexer, encoder]

assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(dataset)
dataset = pipelineModel.transform(dataset)
selectedcols = ['DEPARTURE_DELAY', "features"] 
dataset = dataset.select(selectedcols)
dataset=dataset.select('*').withColumnRenamed('DEPARTURE_DELAY','label')
dataset.printSchema()
dataset.select('features').show()

root
 |-- label: integer (nullable = true)
 |-- features: vector (nullable = true)

+--------------+
|      features|
+--------------+
|[1241.0,292.0]|
| [711.0,292.0]|
|[1357.0,285.0]|
|[1607.0,292.0]|
|[1435.0,292.0]|
| [707.0,278.0]|
|[1724.0,285.0]|
| [819.0,299.0]|
|[2205.0,299.0]|
| [713.0,285.0]|
|[1425.0,285.0]|
|[2218.0,278.0]|
|[1304.0,278.0]|
|[1113.0,285.0]|
|[2052.0,292.0]|
|[1724.0,285.0]|
|[2119.0,292.0]|
|[1956.0,278.0]|
|[1245.0,285.0]|
|[1156.0,285.0]|
+--------------+
only showing top 20 rows



# 3. ML model
## 3.1 generate RDD

In [61]:
# change into RDD
from pyspark.ml.linalg import Vector as MLVector, Vectors as MLVectors
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.ml import linalg as ml_linalg

def as_mllib(v):
    if isinstance(v, ml_linalg.SparseVector):
        return MLLibVectors.sparse(v.size, v.indices, v.values)
    elif isinstance(v, ml_linalg.DenseVector):
        return MLLibVectors.dense(v.toArray())
    else:
        raise TypeError("Unsupported type: {0}".format(type(v)))
        
airlineRDD=dataset.rdd.map(lambda row: LabeledPoint(row['label'],as_mllib(row['features'])))

## 3.2 split trainset and testset 

In [62]:
#  Spliting dataset into train and test dtasets
trainRDD,testRDD=airlineRDD.randomSplit([0.7,0.3])


## 3.3 model training

In [ ]:
# train models
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel
model = LinearRegressionWithSGD.train(trainRDD, iterations=100, step=0.0000001)

## 3.4 model evaluation

In [63]:
# Evaluate the model on training data
valuesAndPreds = testRDD.map(lambda p: (p.label, model.predict(p.features)))
MSE = valuesAndPreds \
    .map(lambda vp: (vp[0] - vp[1])**2) \
    .reduce(lambda x, y: x + y) / valuesAndPreds.count()
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 1302.22631061


##  3.5 save model

In [ ]:
# Save and load model
model.save(sc, "model/pythonLinearRegressionWithSGDModel")

sameModel = LinearRegressionModel.load(sc, "model/pythonLinearRegressionWithSGDModel")